# Preworkout

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.applications import vgg16
from sklearn.decomposition import PCA

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!unzip -q /content/drive/MyDrive/datasets/unbalanced_dataset_2304.zip

# Model

In [ ]:
model = vgg16.VGG16(weights='imagenet', include_top=True)

553476096/553467096 [==============================] - 3s 0us/step


In [ ]:
feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)


# Evaluate

## test dataset

In [ ]:
import utils
GALLERY_PATH = "/content/unbalanced_dataset_2304/validation/gallery/"
QUERY_PATH = "/content/unbalanced_dataset_2304/validation/query/"

target_shape = (224, 224)

gallery = utils.Dataset(GALLERY_PATH, target_shape = target_shape).get_dataset()
query = utils.Dataset(QUERY_PATH, target_shape = target_shape).get_dataset()



Loaded 8777 images from /content/unbalanced_dataset_2304/validation/gallery/ 
Loaded 2236 images from /content/unbalanced_dataset_2304/validation/query/ 


## extract features

In [ ]:
query_features, query_urls, query_labels = utils.compute_features(query, feat_extractor)
pca = PCA(n_components=300)
pca.fit(query_features)
query_pca_features = pca.transform(query_features)

In [ ]:
gallery_features, gallery_urls, gallery_labels = utils.compute_features(gallery, feat_extractor)
pca = PCA(n_components=300)
pca.fit(gallery_features)
gallery_pca_features = pca.transform(gallery_features)

## compute results 

In [ ]:
results = utils.compute_results(query_pca_features, gallery_pca_features, query_urls, gallery_urls)

In [ ]:
utils.evaluate(results, query_labels, gallery_labels)